In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, PatternFill

# 1) Define the new base path and set up DailyArchive folder structure
base_path = Path(r"Z:\work\ach_eobr\Nonspecialty_Vendor_Reports")

# DailyArchive folder under the new base path
daily_archive_folder = base_path / "DailyArchive"
daily_archive_folder.mkdir(parents=True, exist_ok=True)

# Log file stored in DailyArchive
log_file_path = daily_archive_folder / "error_log.txt"

mapping_path = Path(r"Z:\work\ach_eobr\DataSource\VendorList.csv")

# 2) Load data
mapping_data = pd.read_csv(mapping_path)
filtered_data = pd.read_excel('Daily Payments Processed for Vendors (194).xlsx', skiprows=3, header=0)

# Drop the "Unnamed: 0" column if it exists
if "Unnamed: 0" in filtered_data.columns:
    filtered_data = filtered_data.drop("Unnamed: 0", axis=1)

# Convert Check # to str and filter out ones that start with "0"
if "Check #" in filtered_data.columns:
    filtered_data["Check #"] = filtered_data["Check #"].astype(str)
    #filtered_data = filtered_data[~filtered_data["Check #"].str.startswith("0")]

# 3) Define a function to get the previous business day
def get_previous_business_day():
    """Returns the most recent weekday, skipping weekends (if it's Monday, returns Friday)."""
    result = datetime.now()
    if result.weekday() == 0:  # Monday
        result = result - timedelta(days=3)
    else:
        result = result - timedelta(days=1)
    return result

previous_business_day = get_previous_business_day()
formatted_previous_day = previous_business_day.strftime('%Y-%m-%d')
datestamp = previous_business_day.strftime('%Y%m%d')

# Create a date-stamped folder under DailyArchive
date_folder = daily_archive_folder / datestamp
date_folder.mkdir(parents=True, exist_ok=True)

# 4) Filter your data for the previous business day
start_date = formatted_previous_day
end_date = formatted_previous_day

filtered_data = filtered_data[
    (filtered_data['Transaction Date'] >= start_date) &
    (filtered_data['Transaction Date'] <= end_date)
]

# 5) Format columns
filtered_data['Service Start'] = pd.to_datetime(filtered_data['Service Start']).dt.strftime('%Y-%m-%d')
filtered_data['Service End']   = pd.to_datetime(filtered_data['Service End']).dt.strftime('%Y-%m-%d')
filtered_data['Transaction Date'] = pd.to_datetime(filtered_data['Transaction Date']).dt.strftime('%Y-%m-%d')

# Adjust Payee Tax ID for merging
filtered_data['Payee Tax ID'] = filtered_data['Payee Tax ID'].astype(str)
filtered_data['Payee Tax ID'] = filtered_data['Payee Tax ID'].str.split('.').str[0]

mapping_data['VendorTaxID'] = mapping_data['VendorTaxID'].astype(str)
mapping_data['VendorTaxID'] = mapping_data['VendorTaxID'].str.split('.').str[0]

# 6) Merge with mapping_data to find VendorCode
filtered_data = filtered_data.merge(
    mapping_data[['VendorTaxID', 'VendorCode']],
    how='left',
    left_on='Payee Tax ID',
    right_on='VendorTaxID'
)

# Where no match is found, mark it as OTHER_VENDOR_PAYMENTS
filtered_data['VendorCode'].fillna('OTHER_VENDOR_PAYMENTS', inplace=True)

# Drop extra columns if you like
cols_to_drop = ['Payee Tax ID', 'VendorTaxID', 'Check #']
for col in cols_to_drop:
    if col in filtered_data.columns:
        filtered_data.drop(columns=[col], inplace=True)

# 7) Keep only OTHER_VENDOR_PAYMENTS
filtered_data = filtered_data[filtered_data['VendorCode'] == 'OTHER_VENDOR_PAYMENTS']

# If nothing is under OTHER_VENDOR_PAYMENTS, message and exit.
if filtered_data.empty:
    print(f"No OTHER_VENDOR_PAYMENTS found for {formatted_previous_day}. Exiting...")
    exit(0)

# 8) Utility function to sanitize filenames
def sanitize_filename(name):
    return (name.replace('/', '_')
                .replace('\\', '_')
                .replace(':', '_')
                .replace('*', '_')
                .replace('?', '_')
                .replace('"', '_')
                .replace('<', '_')
                .replace('>', '_')
                .replace('|', '_'))

# 9) Export the data by payee directly into the date-stamped folder (no subfolders)
with open(log_file_path, 'w') as log_file:
    for payee, group in filtered_data.groupby('Payee'):
        try:
            sanitized_payee = sanitize_filename(payee)
            file_path = date_folder / f"{sanitized_payee}_transactions_{datestamp}.xlsx"

            group.to_excel(file_path, index=False)

            wb = load_workbook(file_path)
            ws = wb.active

            header_font = Font(bold=True, color="FFFFFF")
            header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
            header_alignment = Alignment(horizontal="center", vertical="center")

            for cell in ws[1]:
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = header_alignment

            for col in ws.columns:
                max_length = 0
                col_letter = get_column_letter(col[0].column)
                for cell in col:
                    if cell.value is not None:
                        max_length = max(max_length, len(str(cell.value)))
                ws.column_dimensions[col_letter].width = max_length + 2


            last_row = ws.max_row + 1
            ws.cell(row=last_row, column=8).value = 'Total'
            if 'Amount' in group.columns:
                ws.cell(row=last_row, column=9).value = group['Amount'].sum()

            wb.save(file_path)

        except Exception as e:
            msg = f"Error processing payee '{payee}': {str(e)}"
            log_file.write(msg + "\n")
            print(msg)

print("OTHER_VENDOR_PAYMENTS process completed successfully.")


C:\Users\gduln001\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\gduln001\AppData\Local\Temp\2\ipykernel_22776\2377010456.py:82: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_data['VendorCode'].fillna('OTHER_VENDOR_PAYMENTS', inplace=True)


OTHER_VENDOR_PAYMENTS process completed successfully.


In [2]:
filtered_data

,Payee,Transaction Date,Claimant Name,Claim Number,Invoice Number,Bill Review Bill ID,Service Start,Service End,Amount,VendorCode
0,FIRST COLONIES ANESTHESIA ASSOCIATE,2025-04-16,"Dawald, Jeffrey R",DLRW2023100505,03X27804312,99997-H-13317558-1,2024-08-30,2024-08-30,110.37,OTHER_VENDOR_PAYMENTS
1,PHOENIX EMERGENCY MEDICINE OF BROWA,2025-04-16,"Monsegue, Shantelle L",DCLW2025004398,7001095059235B,99997-H-13355719-0,2025-03-20,2025-03-20,289.80,OTHER_VENDOR_PAYMENTS
2,FIRSTPATH LLC,2025-04-16,"Monsegue, Shantelle L",DCLW2025004398,122675676P,99997-H-13364274-0,2025-03-22,2025-03-22,4.80,OTHER_VENDOR_PAYMENTS
3,SO CAL SPINE & ORTHO ONC INC,2025-04-16,"Velazquez, Kevin A",DLRW2025104217,VELKE00062396,99997-H-13359901-0,2025-03-19,2025-03-19,102.11,OTHER_VENDOR_PAYMENTS
18,FORT LAUDERDALE OMS PA,2025-04-16,"Audea, Albert",DCLW2025004429,25212,99997-D-13368793-0,2025-04-03,2025-04-03,1452.00,OTHER_VENDOR_PAYMENTS
...,...,...,...,...,...,...,...,...,...,...
362,REGENTS OF THE UNIVERSITY OF,2025-04-16,"Salcido, Brenda A",DLRW2024104164,P919912041,99997-H-13354515-0,2024-11-11,2024-11-11,229.52,OTHER_VENDOR_PAYMENTS
363,ROGER V BERTOLDI MD,2025-04-16,"Clevenger, Velora V",DLRW2023099345,CLEVE00022656,99997-H-13358939-0,2025-03-19,2025-03-19,76.73,OTHER_VENDOR_PAYMENTS
365,THE QUEEN'S MEDICAL CENTER,2025-04-16,"King, Michael E",DLRW2018084117,12466197500,99997-U-13341911-0,2024-12-17,2024-12-17,2353.00,OTHER_VENDOR_PAYMENTS
366,CENTRO MEDICO DE COZUMEL S.A. DE C.,2025-04-16,"De Santiago Arredondo, Rodrigo",DCLW2025004332,226831,99997-Y-13348804-0,2025-02-10,2025-02-10,829.99,OTHER_VENDOR_PAYMENTS
